# Assignment 2
Bradley Thompson - CS 510 LLM Winter 2024

In [13]:
DEFAULT_PROMPT_TEMPLATE = "Of the categories 'positive', 'negative' and 'neutral', the statement '%s' is"
GEN_CONFIG = {
    "min_new_tokens": 1,
    "max_new_tokens": 1,
    "use_cache": False,
}

MODELS = [
    "bigscience/bloom-560m",
    "bigscience/bloom-1b1",
    "bigscience/bloom-1b7",
    "bigscience/bloomz-560m",
    "bigscience/bloomz-1b1",
    "bigscience/bloomz-1b7",
]


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from typing import Dict, Any
import evaluate
import torch as t

perplexity = evaluate.load("perplexity", module_type="metric")

def gen(model, tokenizer, config, prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    gen_config: GenerationConfig = GenerationConfig.from_dict(config)
    return model.generate(inputs, gen_config)

def calculate_metrics(model_name, tokenizer, output):
    print("Raw: ", output)
    input_text = tokenizer.decode(output)
    print("Decoded: ", input_text)
    ttr = len(t.unique(output)) / len(output)
    print("TTR: ", ttr)
    # https://stackoverflow.com/questions/75886674/how-to-compute-sentence-level-perplexity-from-hugging-face-language-models
    results = perplexity.compute(model_id=model_name, add_start_token=False, predictions=[input_text])
    print("Perplexity: ", results["perplexities"][0])
    

In [15]:
model_name = MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [33]:
import numpy as np
categories: np.array = tokenizer.encode("positive negative neutral", return_tensors="pt").numpy()
categories

array([[96675, 23381, 40979]])

In [34]:
categorization_config = GEN_CONFIG | {"force_words_ids": categories.tolist()}

input = "okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry)"
prompt = DEFAULT_PROMPT_TEMPLATE % input

outputs = gen(model, tokenizer, categorization_config, prompt)
calculate_metrics(model_name, tokenizer, outputs[0])

ValueError: `num_beams` has to be an integer strictly greater than 1, but is 1. For `num_beams` == 1, one should make use of `greedy_search` instead.